### Fazendo os imports necessários

In [1]:
import pandas as pd

### Lendo os dados

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/FelipeMMMendes/Machine-Learning-Churn-Project/main/Data/train_churn.csv", delimiter=";")

### Verificando os dados

In [3]:
df.head()

,Unnamed: 0,genero,idoso,parceiro,dependentes,tempoDeServico,ServicoTelefone,MultiLinhas,ServicoInternet,ServicoSegurancaCyber,...,StreamingTV,StreamingFilmes,Contrato,BillingDigital,MetodoPagamento,FaturaMensal,FaturaTotal,NumTickets,NumTicketsTecnico,Churn
0,6027,Masculino,0.0,Sim,Sim,35.0,Sim,Sim,Não,Sem serviço de internet,...,Sem serviço de internet,Sem serviço de internet,2 Anos,Não,Transferência Bancária (Automática),25.45,809.25,0.0,0.0,0.0
1,1961,Feminino,0.0,Não,Não,42.0,Sim,Sim,DSL,NaN,...,Não,Não,1 Ano,Sim,Transferência Bancária (Automática),60.15,2421.6,0.0,0.0,0.0
2,5308,Feminino,0.0,Não,Não,69.0,Sim,Não,DSL,Sim,...,Sim,Sim,2 Anos,Sim,Cartão de Crédito (Automático),82.45,5555.3,0.0,0.0,0.0
3,3587,Masculino,1.0,Não,Não,9.0,Sim,NaN,Fibra óptica,Não,...,Sim,Sim,Mensal,Sim,Cartão de Crédito (Automático),99.45,919.4,0.0,0.0,1.0
4,576,Masculino,0.0,Sim,Sim,31.0,Sim,Sim,Fibra óptica,Sim,...,Sim,Não,1 Ano,Não,Transferência Bancária (Automática),98.05,3082.1,NaN,0.0,0.0


In [4]:
#deletando coluna Unnamed: 0
df.drop("Unnamed: 0",axis=1,inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   genero                 4759 non-null   object 
 1   idoso                  4735 non-null   float64
 2   parceiro               4762 non-null   object 
 3   dependentes            4766 non-null   object 
 4   tempoDeServico         4747 non-null   float64
 5   ServicoTelefone        4789 non-null   object 
 6   MultiLinhas            4745 non-null   object 
 7   ServicoInternet        4771 non-null   object 
 8   ServicoSegurancaCyber  4778 non-null   object 
 9   ServicoBackup          4737 non-null   object 
 10  SeguroDispositivos     4750 non-null   object 
 11  ServicoSuporteTecnico  4767 non-null   object 
 12  StreamingTV            4751 non-null   object 
 13  StreamingFilmes        4775 non-null   object 
 14  Contrato               4738 non-null   object 
 15  Bill

In [9]:
df.describe()

,idoso,tempoDeServico,FaturaMensal,NumTickets,NumTicketsTecnico,Churn
count,4735.000000,4747.000000,4726.000000,4731.000000,4745.000000,4736.000000
mean,0.163464,32.158205,64.801947,0.503488,0.411170,0.263091
std,0.369827,29.592578,35.930051,1.260113,1.230994,0.440358
min,0.000000,-112.000000,-127.000000,0.000000,0.000000,0.000000
25%,0.000000,8.000000,34.700000,0.000000,0.000000,0.000000
50%,0.000000,29.000000,70.400000,0.000000,0.000000,0.000000
75%,0.000000,56.000000,90.400000,0.000000,0.000000,1.000000
max,1.000000,190.000000,247.000000,5.000000,8.000000,1.000000
